In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Utils.DataLoader import DataLoader
from Utils.DataTransformer import DataTransformer
from Utils.DuckDb.DuckDb import DuckDb
from Utils.DatasetProcessor.DatasetProcessorUtils import DatasetProcessorUtils
from datetime import datetime
import pandas as pd
from typing import List
from Utils import PandasNotebookConfigs
import project_config
import numpy as np
import matplotlib.pyplot as plt
import joblib
from datetime import datetime
from Utils.Datetime import DatetimeUtils


In [3]:
PandasNotebookConfigs.config()

Ajustando display.float.format para %.4f
Ajustando prints de linhas e colunas


In [4]:
DUCK_DB = DuckDb()
DATA_TRANSFORMER = DataTransformer()
DATA_PROCESSOR_UTILS = DatasetProcessorUtils(DUCK_DB)
DATETIME_UTILS = DatetimeUtils()

In [5]:
PRE_PREPARED_DATA_TABLE = project_config.PRE_PREPARED_DATA_TABLE
PREDICTED_DATA_TABLE = f'predicted_dataset_{DATETIME_UTILS.get_datetime_string_identifier()}'

In [6]:
PREDICTED_DATA_TABLE

'predicted_dataset_2025_04_05_08h07m'

In [7]:
# df = DUCK_DB.load_table_to_predict(
#     PRE_PREPARED_DATA_TABLE,
#     start_from_safra=project_config.MAX_SAFRA_TO_CONSIDER_ON_DATA_PREPARE
# )

df = DUCK_DB.load_table(
    PRE_PREPARED_DATA_TABLE,
    where_condition=f'WHERE safra > {project_config.TRAIN_DATA_UNTIL_SAFRA}'
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
# df = df[:10].copy()

In [9]:
# df.sort_values('safra', ascending=False).head()[['safra', 'is_churn', 'no_churn_information']]

In [10]:
# df[df['no_churn_information'] == True]['safra'].describe()

In [11]:
df = DATA_PROCESSOR_UTILS.calc_remaining_days(df)

In [12]:
df.head()

,cost,msno,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,msno_1,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,safra_1,transaction_date_year,transaction_date_month,transaction_date_day,transaction_date_day_of_week,transaction_date_day_of_year,membership_expire_date_year,membership_expire_date_month,membership_expire_date_day,membership_expire_date_day_of_week,membership_expire_date_day_of_year,discount,price_per_month,members_msno,members_safra,city,registered_via,is_active,registration_init_time_year,registration_init_time_month,registration_init_time_day,registration_init_time_day_of_week,registration_init_time_day_of_year,_filled_out_members_info,is_churn,no_churn_information,num_unq-2M,num_unq-1M,total_secs-2M,total_secs-1M,num_25-2M,num_25-1M,num_50-2M,num_50-1M,num_75-2M,num_75-1M,num_985-2M,num_985-1M,num_100-2M,num_100-1M,remaining_days
0,55.9125,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,201609,60,12,14,7,171,179,49996,13.8878,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,35,7,0,0,False,False,201609,2016,9,9,4,253,2016,9,14,2,258,0,0.0000,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,201609,15,9,True,2004,3,27,5,87,Infos já existentes,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0000
1,67.3556,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201611,128,71,50,101,470,488,148668,41.2967,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,False,201611,2016,11,16,2,321,2016,12,15,3,350,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201611,1,7,True,2014,6,8,6,159,Infos já existentes,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0000
2,80.6741,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201612,193,88,98,142,855,979,256812,71.3367,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,False,201612,2016,12,15,3,350,2017,1,15,6,15,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201612,1,7,True,2014,6,8,6,159,Infos já existentes,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0000
3,69.3364,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201701,135,69,78,147,438,696,157868,43.8522,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,False,201701,2017,1,15,6,15,2017,2,15,2,46,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,<NA>,1,7,True,2014,6,8,6,159,Copiadas,False,True,488.0000,979.0000,148668.0000,256812.0000,128.0000,193.0000,71.0000,88.0000,50.0000,98.0000,101.0000,142.0000,470.0000,855.0000,31.0000
4,68.9275,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201702,116,61,58,114,468,642,156533,43.4814,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,False,201702,2017,2,15,2,46,2017,3,15,2,74,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,<NA>,1,7,True,2014,6,8,6,159,Copiadas,False,True,979.0000,696.0000,256812.0000,157868.0000,193.0000,135.0000,88.0000,69.0000,98.0000,78.0000,142.0000,147.0000,855.0000,438.0000,28.0000


In [13]:
# Data to predict
df['safra'].value_counts()

safra
201611    227022
201610    219759
201609    213338
201612    211127
201701    205849
201702    173975
Name: count, dtype: int64

In [14]:
len(df)

1251070

In [15]:
# Treating DF
df.isna().sum().T

cost                                       0
msno                                       0
safra                                      0
num_25                                     0
num_50                                     0
num_75                                     0
num_985                                    0
num_100                                    0
num_unq                                    0
total_secs                                 0
total_hours                                0
msno_1                                     0
payment_method_id                          0
payment_plan_days                          0
plan_list_price                            0
actual_amount_paid                         0
is_auto_renew                              0
is_cancel                                  0
safra_1                                    0
transaction_date_year                      0
transaction_date_month                     0
transaction_date_day                       0
transactio

In [16]:
count = 0

def fill_out_historical_data(df: pd.DataFrame) -> pd.DataFrame:
    past_safras_to_consider = project_config.SAFRAS_TO_CONSIDER_ON_HISTORICAL_DATA

    # Key: past col
    # Value: "actual" col
    historical_cols = {}

    for col in project_config.COLUMNS_TO_GET_HISTORICAL_DATA:
        for s in past_safras_to_consider:
            historical_cols[f'{col}{s}M'] = col

    df_size = len(df)

    def __process(row: pd.Series):
        global count
        if count % 1500 == 0:
            print(f'-> {count} / {df_size}')

        count += 1

        edited = False
        for past_col, col in historical_cols.items():
            if pd.isna(row[past_col]):
                row[past_col] = int(row[col])
                edited = True

        row['_filled_out_past_info'] = edited

        return row

    df = df.apply(lambda row: __process(row), axis=1)

    return df

df = fill_out_historical_data(df)

-> 0 / 1251070
-> 1500 / 1251070
-> 3000 / 1251070
-> 4500 / 1251070
-> 6000 / 1251070
-> 7500 / 1251070
-> 9000 / 1251070
-> 10500 / 1251070
-> 12000 / 1251070
-> 13500 / 1251070
-> 15000 / 1251070
-> 16500 / 1251070
-> 18000 / 1251070
-> 19500 / 1251070
-> 21000 / 1251070
-> 22500 / 1251070
-> 24000 / 1251070
-> 25500 / 1251070
-> 27000 / 1251070
-> 28500 / 1251070
-> 30000 / 1251070
-> 31500 / 1251070
-> 33000 / 1251070
-> 34500 / 1251070
-> 36000 / 1251070
-> 37500 / 1251070
-> 39000 / 1251070
-> 40500 / 1251070
-> 42000 / 1251070
-> 43500 / 1251070
-> 45000 / 1251070
-> 46500 / 1251070
-> 48000 / 1251070
-> 49500 / 1251070
-> 51000 / 1251070
-> 52500 / 1251070
-> 54000 / 1251070
-> 55500 / 1251070
-> 57000 / 1251070
-> 58500 / 1251070
-> 60000 / 1251070
-> 61500 / 1251070
-> 63000 / 1251070
-> 64500 / 1251070
-> 66000 / 1251070
-> 67500 / 1251070
-> 69000 / 1251070
-> 70500 / 1251070
-> 72000 / 1251070
-> 73500 / 1251070
-> 75000 / 1251070
-> 76500 / 1251070
-> 78000 / 1251070
-> 

In [17]:
def print_cols_and_historical_cols(df: pd.DataFrame):
    cols = ['_filled_out_past_info']
    for col in project_config.COLUMNS_TO_GET_HISTORICAL_DATA:
        cols.append(col)

        for sfr in project_config.SAFRAS_TO_CONSIDER_ON_HISTORICAL_DATA:
            cols.append(f'{col}{sfr}M')

    return df[cols]

print_cols_and_historical_cols(df).head().T

,0,1,2,3,4
_filled_out_past_info,True,True,True,False,False
num_unq,179,488,979,696,642
num_unq-2M,179.0000,488.0000,979.0000,488.0000,979.0000
num_unq-1M,179.0000,488.0000,979.0000,979.0000,696.0000
total_secs,49996,148668,256812,157868,156533
total_secs-2M,49996.0000,148668.0000,256812.0000,148668.0000,256812.0000
total_secs-1M,49996.0000,148668.0000,256812.0000,256812.0000,157868.0000
num_25,60,128,193,135,116
num_25-2M,60.0000,128.0000,193.0000,128.0000,193.0000
num_25-1M,60.0000,128.0000,193.0000,193.0000,135.0000


In [18]:
# Treating DF
df.isna().sum().T

cost                                       0
msno                                       0
safra                                      0
num_25                                     0
num_50                                     0
num_75                                     0
num_985                                    0
num_100                                    0
num_unq                                    0
total_secs                                 0
total_hours                                0
msno_1                                     0
payment_method_id                          0
payment_plan_days                          0
plan_list_price                            0
actual_amount_paid                         0
is_auto_renew                              0
is_cancel                                  0
safra_1                                    0
transaction_date_year                      0
transaction_date_month                     0
transaction_date_day                       0
transactio

In [19]:
df = DATA_PROCESSOR_UTILS.process_categories(df)

Adicionando coluna payment_method_id_0
Adicionando coluna payment_method_id_1
Adicionando coluna payment_method_id_2
Adicionando coluna payment_method_id_3
Adicionando coluna payment_method_id_4
Adicionando coluna payment_method_id_5
Adicionando coluna payment_method_id_7
Adicionando coluna payment_method_id_9
Adicionando coluna payment_method_id_24
Adicionando coluna payment_method_id_25
Adicionando coluna city_0
Adicionando coluna city_2
Adicionando coluna registered_via_0
Adicionando coluna registered_via_1
Adicionando coluna registered_via_2
Adicionando coluna registered_via_5
Adicionando coluna registered_via_6
Adicionando coluna registered_via_8
Adicionando coluna registered_via_10
Adicionando coluna registered_via_11
Adicionando coluna registered_via_12
Adicionando coluna registered_via_14
Adicionando coluna registered_via_15
Adicionando coluna registered_via_16
Adicionando coluna registered_via_17
Adicionando coluna registered_via_18
Adicionando coluna registered_via_19


In [20]:
df = DATA_PROCESSOR_UTILS.split_safras(df)

In [21]:
df_to_predict = df[project_config.COLUMNS_USED_ON_TRAIN_AND_PREDICTION]

In [22]:
df_to_predict.head()

,registration_init_time_day,registration_init_time_day_of_year,registration_init_time_year,transaction_date_day,transaction_date_day_of_week,transaction_date_day_of_year,transaction_date_month,membership_expire_date_day,membership_expire_date_day_of_year,membership_expire_date_month,membership_expire_date_year,is_auto_renew,is_cancel,remaining_days,safra_month,num_100,num_100-1M,num_100-2M,num_25,num_25-1M,num_25-2M,num_50,num_50-1M,num_50-2M,num_75,num_75-1M,num_75-2M,num_985,num_985-1M,num_985-2M,num_unq,num_unq-1M,num_unq-2M,total_secs,total_secs-1M,total_secs-2M,city_0,city_1,city_10,city_11,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_2,city_20,city_21,city_22,city_3,city_4,city_5,city_6,city_7,city_8,city_9,registered_via_0,registered_via_1,registered_via_10,registered_via_11,registered_via_12,registered_via_13,registered_via_14,registered_via_15,registered_via_16,registered_via_17,registered_via_18,registered_via_19,registered_via_2,registered_via_3,registered_via_4,registered_via_5,registered_via_6,registered_via_7,registered_via_8,registered_via_9,payment_method_id_0,payment_method_id_1,payment_method_id_10,payment_method_id_11,payment_method_id_12,payment_method_id_13,payment_method_id_14,payment_method_id_15,payment_method_id_16,payment_method_id_17,payment_method_id_18,payment_method_id_19,payment_method_id_2,payment_method_id_20,payment_method_id_21,payment_method_id_22,payment_method_id_23,payment_method_id_24,payment_method_id_25,payment_method_id_26,payment_method_id_27,payment_method_id_28,payment_method_id_29,payment_method_id_3,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_4,payment_method_id_40,payment_method_id_41,payment_method_id_5,payment_method_id_6,payment_method_id_7,payment_method_id_8,payment_method_id_9
0,27,87,2004,9,4,253,9,14,258,9,2016,False,False,5.0000,9.0000,171,171.0000,171.0000,60,60.0000,60.0000,12,12.0000,12.0000,14,14.0000,14.0000,7,7.0000,7.0000,179,179.0000,179.0000,49996,49996.0000,49996.0000,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,8,159,2014,16,2,321,11,15,350,12,2016,True,False,29.0000,11.0000,470,470.0000,470.0000,128,128.0000,128.0000,71,71.0000,71.0000,50,50.0000,50.0000,101,101.0000,101.0000,488,488.0000,488.0000,148668,148668.0000,148668.0000,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,8,159,2014,15,3,350,12,15,15,1,2017,True,False,31.0000,12.0000,855,855.0000,855.0000,193,193.0000,193.0000,88,88.0000,88.0000,98,98.0000,98.0000,142,142.0000,142.0000,979,979.0000,979.0000,256812,256812.0000,256812.0000,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [23]:
from sklearn.ensemble import RandomForestClassifier

model: RandomForestClassifier = joblib.load(project_config.SELECTED_MODEL)
print(f'Carregado modelo {project_config.SELECTED_MODEL}: {model}')

Carregado modelo ./models/random_forest_model_2025-04-05-08h-05m.joblib: RandomForestClassifier(bootstrap=False, max_depth=30, max_features='log2',
                       min_samples_split=10, random_state=42)


In [24]:
predicted_is_churn = model.predict(df_to_predict)
predicted_is_churn_proba = model.predict_proba(df_to_predict)

In [25]:
predicted_is_churn[:5], predicted_is_churn_proba[:5, 0]

(array([ True, False, False, False, False]),
 array([0.36326286, 0.79440992, 0.56946953, 0.58379437, 0.57548709]))

In [26]:
df['predicted_is_churn'] = predicted_is_churn

df['predicted_is_churn_proba_false'] = predicted_is_churn_proba[:, 0]
df['predicted_is_churn_proba_true'] = predicted_is_churn_proba[:, 1]

In [27]:
df.head()

,cost,msno,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,msno_1,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,safra_1,transaction_date_year,transaction_date_month,transaction_date_day,transaction_date_day_of_week,transaction_date_day_of_year,membership_expire_date_year,membership_expire_date_month,membership_expire_date_day,membership_expire_date_day_of_week,membership_expire_date_day_of_year,discount,price_per_month,members_msno,members_safra,is_active,registration_init_time_year,registration_init_time_month,registration_init_time_day,registration_init_time_day_of_week,registration_init_time_day_of_year,_filled_out_members_info,is_churn,no_churn_information,num_unq-2M,num_unq-1M,total_secs-2M,total_secs-1M,num_25-2M,num_25-1M,num_50-2M,num_50-1M,num_75-2M,num_75-1M,num_985-2M,num_985-1M,num_100-2M,num_100-1M,remaining_days,_filled_out_past_info,payment_method_id_6,payment_method_id_8,payment_method_id_10,payment_method_id_11,payment_method_id_12,payment_method_id_13,payment_method_id_14,payment_method_id_15,payment_method_id_16,payment_method_id_17,payment_method_id_18,payment_method_id_19,payment_method_id_20,payment_method_id_21,payment_method_id_22,payment_method_id_23,payment_method_id_26,payment_method_id_27,payment_method_id_28,payment_method_id_29,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41,city_1,city_10,city_11,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22,city_3,city_4,city_5,city_6,city_7,city_8,city_9,registered_via_13,registered_via_3,registered_via_4,registered_via_7,registered_via_9,payment_method_id_0,payment_method_id_1,payment_method_id_2,payment_method_id_3,payment_method_id_4,payment_method_id_5,payment_method_id_7,payment_method_id_9,payment_method_id_24,payment_method_id_25,city_0,city_2,registered_via_0,registered_via_1,registered_via_2,registered_via_5,registered_via_6,registered_via_8,registered_via_10,registered_via_11,registered_via_12,registered_via_14,registered_via_15,registered_via_16,registered_via_17,registered_via_18,registered_via_19,safra_year,safra_month,predicted_is_churn,predicted_is_churn_proba_false,predicted_is_churn_proba_true
0,55.9125,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,201609,60,12,14,7,171,179,49996,13.8878,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7,0,0,False,False,201609,2016,9,9,4,253,2016,9,14,2,258,0,0.0000,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,201609,True,2004,3,27,5,87,Infos já existentes,True,False,179.0000,179.0000,49996.0000,49996.0000,60.0000,60.0000,12.0000,12.0000,14.0000,14.0000,7.0000,7.0000,171.0000,171.0000,5.0000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2016.0000,9.0000,True,0.3633,0.6367
1,67.3556,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201611,128,71,50,101,470,488,148668,41.2967,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30,99,99,True,False,201611,2016,11,16,2,321,2016,12,15,3,350,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201611,True,2014,6,8,6,159,Infos já existentes,False,False,488.0000,488.0000,148668.0000,148668.0000,128.0000,128.0000,71.0000,71.0000,50.0000,50.0000,101.0000,101.0000,470.0000,470.0000,29.0000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [28]:
DUCK_DB.create_database_table(
    df, PREDICTED_DATA_TABLE
)

In [29]:
DUCK_DB.truncate_table(
    PREDICTED_DATA_TABLE
)

In [30]:
DUCK_DB.upload_dataframe_to_duck_db(
    df, PREDICTED_DATA_TABLE
)

2025-04-05 08:14:59.954284 Inserindo 1251070 registros na tabela predicted_dataset_2025_04_05_08h07m


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-04-05 08:16:43.550890 Inseridos com sucesso


In [31]:
# DUCK_DB_UTILS.get_connection().close()

In [32]:
DUCK_DB.commit()